### This notebook aggregates the underlying data for the figures and the tables
### I addition, it also generates files for some of the figures.
#### <span style='color:red'>Note that this notebook **do not** create the results of the nested cross validation pipeline</span> 

In [ ]:
import importlib

import os
os.chdir('../')
from multiprocessing import Pool
import numpy as np
import pandas as pd
import constants
import utils

from aggregate_prs_statistics_cv_generic import aggregate_statistics_cv_mp
from aggregate_prs_statistics_cv_generic import aggregate_statistics_cv
from aggregate_folds_cv_generic import aggregate_folds_cv_multi_test
from choose_model import choose_model

In [ ]:
# This function faciliates multi-process execution of aggregate_statistics_cv
def aggregate_statistics_cv_mp(args):
    discoveries, targets, imps, method, hyperparameters, cv_folds, rep, suffix = args
    aggregate_statistics_cv(discoveries, targets, imps, method, hyperparameters, cv_folds, rep, suffix)

### Parameter defintions: choose the parameters according to the figure that you wish to produce

#### Figure 3 + S1 + S2

In [ ]:
discoveries = [f"UKB_{a}_eur" for a in ["ctrt", "chol", "hfvr", "hyty", "ht", "osar", "madd", "t2d", "ast", "utfi", "gerx", "angna"]]
targets = ["ukbb_sas", 'ukbb_afr'] 
imps = ["impute2_1kg_eur","impute2_1kg_sas","impute2_1kg_afr"]

methods=["pt2", "pt3", "ls"] 
suffix="ukb_eur"

field_names=["or.all", "or.90"]

rep_start="105_1"
rep_end="105_6"

#### Table 4

In [ ]:
discoveries = ["D2_sysp_evangelou_2018", "D2_ldlp_willer_2013", "D2_t2di_mahajan_2018", "D2_gerx_an_2019", "D2_madd_howard_2019"]
targets = ["ukbb_sas", "ukbb_afr"]
imps = ["impute2_1kg_eur","impute2_1kg_eas","impute2_1kg_afr"] 

methods=["pt2", "pt3", "ls"] 
suffix="public_eur"

field_names=["or.all", "or.90"]

rep_start="105_1"
rep_end="105_6"

#### Table 5

In [ ]:
discoveries = ["PGC2_noAJ"]
targets = ["dbg-scz19", "gain_afr"]
imps = ["impute2_1kg_eur","impute2_1kg_eas","impute2_1kg_afr","impute2_1kg_eur2","impute2_1kg_eas2","impute2_1kg_afr2"] 

methods=["pt2", "pt3", "ls"] 
suffix="scz_eur"

field_names=["or.all", "or.90"]

rep_start="105_1"
rep_end="105_6"

#### Figure 4

In [ ]:
discoveries = ["PGC2_noAJ"]
targets = ["dbg-scz19"]
imps = ["impute2_ajkg14_t101","impute2_1kg_ceu2","impute2_1kg_gbr2","impute2_1kg_fin2","impute2_1kg_ibs2","impute2_1kg_tsi2"] 

methods=["pt2", "pt3", "ls"] 
suffix="scz_aj"

field_names=["or.all", "or.90"]

rep_start="105_1"
rep_end="105_6"

#### Figure S2

In [ ]:
discoveries = [f"UKB_{a}_eur" for a in ["ctrt", "chol", "hfvr", "hyty", "ht", "osar", "madd", "t2d", "ast", "utfi", "gerx", "angna"]]
targets = ["ukbb_sas"] 
imps = ["impute2_1kg_eur","impute2_1kg_sas","impute2_1kg_afr"]

methods=["ld"] 
suffix="ukb_eur_ld"

field_names=["or.all", "or.90"]

rep_start="105_1"
rep_end="105_6"

#### Figure S3

In [ ]:
discoveries = [f"GC_{a}_sakaue_2021" for a in ["ctrt", "chol", "hfvr", "hyty", "ht", "osar", "madd", "t2d", "ast", "utfi", "gerx", "angna"]]
targets = ["ukbb_sas", 'ukbb_afr'] 
imps = ["impute2_1kg_eur","impute2_1kg_sas","impute2_1kg_afr"]

methods=["pt2", "pt3", "ls"] 
suffix="gc_eas"

field_names=["or.all", "or.90"]

rep_start="105_1"
rep_end="105_6"

#### Figure S4


In [ ]:
discoveries = [f"UKB_{a}_eur" for a in ["ctrt", "chol", "hfvr", "hyty", "ht", "osar", "madd", "t2d", "ast", "utfi", "gerx", "angna"]]
targets = ["ukbb_sas", 'ukbb_afr'] 
imps = ["impute2_1kg_eur", "impute2_1kg_eur100"]

methods=["pt2", "pt3", "ls"] 
suffix="ukb_eur100"

field_names=["or.all", "or.90"]

rep_start="105_1"
rep_end="105_6"

#### Table S6

In [ ]:
discoveries = ["LH_PGC-SCZ-EAS", "LH_PGC-SCZ-EAS_LS"]
targets = ["dbg-scz19", "gain_afr"]
imps = ["impute2_1kg_eur","impute2_1kg_eas","impute2_1kg_afr","impute2_1kg_eur2","impute2_1kg_eas2","impute2_1kg_afr2"] 

methods=["pt2", "pt3", "ls"] 
suffix="scz_eas"

field_names=["or.all", "or.90"]

rep_start="105_1"
rep_end="105_6"

### Aggregate the data obtanied from all CV

#### aggregate_prs_statistics_cv aggregate_folds_cv

In [ ]:
from aggregate_prs_statistics_cv_generic import aggregate_statistics_cv
from aggregate_folds_cv_generic import aggregate_folds_cv_multi_test

# file_name_formats=["prs.cv.{}{{{}}}.statistics_summary_{{}}{{{}}}.tsv", "prs.cv.pt3{{}}.statistics_summary_{}{{}}.tsv", "prs.cv.ls{{}}.statistics_summary_{}{{}}.tsv"] # ["prs.cv.ld{{}}.statistics_summary_{}{{}}.tsv"]

for method in methods: # ["pt2", "pt3", "ls", "ld"]:
    base_rep = 105
    n_reps = 6
    cv_folds=5
    if method == "ls":
        hyperparameters = [f'{a}-{b}' for a in [0.2, 0.5, 0.9, 1] for b in range(1, 21)]
    elif method == "ld" and not discoveries[0]=="PGC2_noAJ":
        hyperparameters=[str(a) for a in range(1,103)]
        base_rep = 102
        cv_folds=2
        n_reps = 3
    else:
        hyperparameters = ["0.00000005", "0.00G00001", "0.000001", "0.00001", "0.0001", "0.001", "0.005", "0.01",
                           "0.05", "0.1", "0.2", "0.3", "0.4", "0.5"]
    
    rep_start=f"{base_rep}_1"
    rep_end=f"{base_rep}_{n_reps}"
    
    ## aggregate_prs ##
    for cur_rep in range(1, n_reps + 1):
        rep = f"{base_rep}_{cur_rep}"
        aggregate_statistics_cv_params.append(
            [discoveries, targets, imps, method, hyperparameters, cv_folds, rep, f'{suffix}_{rep}'])
        aggregate_statistics_cv_mp(
            [discoveries, targets, imps, method, hyperparameters, cv_folds, rep, f'{suffix}_{rep}'])
    
    # aggregate_folds ##
    cur_file_name_format="prs.cv.{}{{}}.statistics_summary_{}{{}}.tsv"
    aggregate_folds_cv_multi_test(cur_file_name_format.format(method, suffix), field_names, discoveries, targets, imps=imps, out_suffix=suffix, rep_start=rep_start, rep_end=rep_end)
        
#     with Pool(50) as p:
#         print(p.map(aggregate_statistics_cv_mp, aggregate_statistics_cv_params))



 ### Choose the best parameters for the outer folds

#### choose_params

In [ ]:
import choose_model
importlib.reload(choose_model)
from choose_model import choose_model

metrics=['or.all','or.90']
choose_model(methods, rep_start, rep_end, metrics, suffix)

### Plott the figures: choose the parameters according the the figure that you wish to produce

#### plot_by_imputation_panel

In [ ]:
import plot_by_imputation_panel
importlib.reload(plot_by_imputation_panel)
from plot_by_imputation_panel import plot_or_by_imputation

file_name_format="prs.cv.choose_params_{}_{}.tsv"

#### Figures 3A

In [ ]:
targets=targets
imps_figure = imps
methods_figure = ['ls']
suffix_figure = f"all_{suffix}_super_pop"
plot_or_by_imputation(file_name_format.format(suffix, f'{rep_start}_{rep_end}'),
                      prs_names="", discoveries=discoveries, targets=targets_figure, imps=imps_figure,
                      methods=methods_figure, suffix=suffix_figure)

#### Figures 3B,S1: SAS results

In [ ]:
targets_figure = ["ukbb_sas"]
imps_figure = ["original", "impute2_1kg_eur", "impute2_1kg_sas"]
methods_figure = ['pt2', 'pt3']
suffix_figure = f"{target}_{suffix}_super_pop"
plot_or_by_imputation(file_name_format.format(suffix, f'{rep_start}_{rep_end}'), 
                      prs_names="", discoveries=discoveries, targets=targets_figure, imps=imps_figure, 
                      methods=methods_figure, suffix=suffix_figure)

#### Figures 3C, S1: AFR results

In [ ]:
targets_figure = ["ukbb_afr"]  
imps_figure = ["original", "impute2_1kg_eur", "impute2_1kg_afr"] 
methods_figure = ['pt2', 'pt3']
suffix_figure = f"{target}_{suffix}_super_pop"
plot_or_by_imputation(file_name_format.format(suffix, f'{rep_start}_{rep_end}'),
                      prs_names="", discoveries=discoveries, targets=targets_figure, imps=imps_figure,
                      methods=methods, methods_figure=suffix_figure)

#### Figure 4

In [ ]:
targets_figure = targets
imps_figure = imps
methods_figure = methods
suffix_figure = f"{target}_{suffix}_pop"
plot_or_by_imputation(file_name_format.format(suffix, f'{rep_start}_{rep_end}'), 
                      prs_names="", discoveries=discoveries, targets=targets_figure, 
                      imps=imps_figure, methods=methods, suffix=suffix_figure)

#### Figure S2

In [ ]:
targets_figure = targets
imps_figure = imps
methods_figure = methods
suffix_figure = f"{target}_{suffix}_ld_super_pop"
plot_or_by_imputation(file_name_format.format(suffix, f'{rep_start}_{rep_end}'),
                      prs_names="", discoveries=discoveries, targets=targets_figure, imps=imps_figure,
                      methods=methods, suffix=suffix_figure)